Start Project: look at data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
train = pd.read_csv("train.csv")

In [3]:
import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")
PAD_SIZE = 255

In [13]:
def row_to_tokens(row):
    tokens = tokenizer(row)
    if len(tokens) > PAD_SIZE:
        return tokens[:PAD_SIZE] + ["<eos>"]
    else:
        size = PAD_SIZE - len(tokens)
        return tokens + ["<eos>"] + ["<pad>"]*size
    
    
    
    

In [14]:
def get_tokenize_data(X):
    X["comment_text"] = X["comment_text"].apply(row_to_tokens)
    return X

In [15]:
train_X = get_tokenize_data(train)
train_X

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"[explanation, why, the, edits, made, under, my...",0,0,0,0,0,0
1,000103f0d9cfb60f,"[d, ', aww, !, he, matches, this, background, ...",0,0,0,0,0,0
2,000113f07ec002fd,"[hey, man, ,, i, ', m, really, not, trying, to...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"[more, i, can, ', t, make, any, real, suggesti...",0,0,0,0,0,0
4,0001d958c54c6e35,"[you, ,, sir, ,, are, my, hero, ., any, chance...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,"[and, for, the, second, time, of, asking, ,, w...",0,0,0,0,0,0
159567,ffea4adeee384e90,"[you, should, be, ashamed, of, yourself, that,...",0,0,0,0,0,0
159568,ffee36eab5c267c9,"[spitzer, umm, ,, theres, no, actual, article,...",0,0,0,0,0,0
159569,fff125370e4aaaf3,"[and, it, looks, like, it, was, actually, you,...",0,0,0,0,0,0


In [27]:
train_X.loc[0,"comment_text"]

['explanation',
 'why',
 'the',
 'edits',
 'made',
 'under',
 'my',
 'username',
 'hardcore',
 'metallica',
 'fan',
 'were',
 'reverted',
 '?',
 'they',
 'weren',
 "'",
 't',
 'vandalisms',
 ',',
 'just',
 'closure',
 'on',
 'some',
 'gas',
 'after',
 'i',
 'voted',
 'at',
 'new',
 'york',
 'dolls',
 'fac',
 '.',
 'and',
 'please',
 'don',
 "'",
 't',
 'remove',
 'the',
 'template',
 'from',
 'the',
 'talk',
 'page',
 'since',
 'i',
 "'",
 'm',
 'retired',
 'now',
 '.',
 '89',
 '.',
 '205',
 '.',
 '38',
 '.',
 '27',
 '<eos>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>'

In [25]:
vocab = torchtext.vocab.build_vocab_from_iterator(train_X["comment_text"],min_freq=5,specials=["<unk>","<pd>","<eos>"],special_first=True)
vocab.set_default_index(vocab["<unk>"])

In [20]:
import torch
import torch.nn

def collate_batch(batch):
    data = [],labels = []
    for item in batch:
        data.append(vocab(item["comment_text"]))
        labels.append(item[:item.columns[2:]])
    return torch.ToTensor(data,dtype=torch.int64,device=device),torch.ToTensor(labels,dtype=torch.int64,device=device)
    
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-20-4a00641ec04a>, line 1)

we have the training data. now we would like to get some embedding for the words. I will try to use self attention to create context rich embeddings

In [23]:
class SentenceEmbeding(torch.nn.Module):
    
    def __init__(self,vocab_size,output_size):
        self.embed = torch.nn.Embeding(vocab_size,256)
        self.atten = torch.nn.MultiheadAttention(256,1)
        self.linear = torch.nn.Linear(256,128)
        
    def forward(self,x):
        x = self.embed(x)
        x = self.atten(x)
        return self.linear(x)
        